In [ ]:
from hyper_embed import hyperclovaEmbedding
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.embeddings.base import Embeddings

#가정은 Document 객체가 이미 생성되어 있다고 가정하고 진행합니다.

class PrecomputedEmbeddings(Embeddings):
    def __init__(self, embeddings, hyper_clova_instance):
        self._embeddings = embeddings
        self._hyper_clova = hyper_clova_instance
        
    def embed_documents(self, texts):
        return self._embeddings
    
    def embed_query(self, text):
        request_data = {"text": text}
        return self._hyper_clova.execute(request_data)

# hyperclovaEmbedding 인스턴스 생성
hyper_clova = hyperclovaEmbedding(
    host='clovastudio.apigw.ntruss.com',
    api_key='your-api-key',
    api_key_primary_val='your-api-key-primary-val',
    request_id='your-request-id',
)

embeddings = [hyper_clova.execute({"text": doc.page_content}) for doc in docs]

# 임베딩 함수 초기화
embedding_func = PrecomputedEmbeddings(embeddings, hyper_clova)

# FAISS 인덱스 생성
vectorstore = FAISS.from_documents(docs, embedding_func)

# 검색 수행
query = "클라우드 네이티브 사업의 내용을 설명하세요."
search_results = vectorstore.similarity_search_with_score(query, k=5)

# 결과 출력
for doc, score in search_results:
    print(f"Score: {score}")
    print(f"Content: {doc.page_content}")
    print(f"Metadata: {doc.metadata}")
    print("---")